<a href="https://colab.research.google.com/github/junkyuhufs/Practice/blob/main/CL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Corpus Lingusitics Practice
Kyle 홈피 
### https://github.com/kristopherkyle/corpus_toolkit

데이터 마운트

corpus-toolkit설치

In [ ]:
pip install corpus-toolkit

에러발생 제거

In [ ]:
!pip install spacytextblob
!python -m textblob.download_corpora
!python -m spacy download en_core_web_sm

Load, tokenize, and generate a frequency list

In [ ]:
from corpus_toolkit import corpus_tools as ct
brown_corp = ct.ldcorpus("brown_single") #load and read corpus
tok_corp = ct.tokenize(brown_corp) #tokenize corpus - by default this lemmatizes as well
brown_freq = ct.frequency(tok_corp) #creates a frequency dictionary
#note that range can be calculated instead of frequency using the argument calc = "range"
ct.head(brown_freq, hits = 10) #print top 10 items